## STEP 1: COLLECT DATA

### Importing packages and CSV files into corresponding dataframes

In [5]:
import pandas as pd
import plotly.express as px

In [6]:
q2_2019 = pd.read_csv('Divvy_Trips_2019_Q2.csv')
q3_2019 = pd.read_csv("Divvy_Trips_2019_Q3.csv")
q4_2019 = pd.read_csv("Divvy_Trips_2019_Q4.csv")
q1_2020 = pd.read_csv("Divvy_Trips_2020_Q1.csv")

## STEP 2: PREPARE

### Rename columns to create uniformity and for merging later on

In [7]:
q4_2019.rename(columns={'trip_id':'ride_id',
                        'bikeid':'rideable_type',
                        'start_time':'started_at',
                        'end_time':'ended_at',
                        'from_station_name':'start_station_name',
                        'from_station_id':'start_station_id',
                        'to_station_name':'end_station_name',
                        'to_station_id':'end_station_id',
                        'usertype':'member_casual'}, inplace=True)

In [8]:
q3_2019.rename(columns={'trip_id':'ride_id',
                        'bikeid':'rideable_type',
                        'start_time':'started_at',
                        'end_time':'ended_at',
                        'from_station_name':'start_station_name',
                        'from_station_id':'start_station_id',
                        'to_station_name':'end_station_name',
                        'to_station_id':'end_station_id',
                        'usertype':'member_casual'}, inplace=True)

In [9]:
q2_2019.rename(columns={'01 - Rental Details Rental ID':'ride_id',
                        '01 - Rental Details Bike ID':'rideable_type',
                        '01 - Rental Details Local Start Time':'started_at',
                        '01 - Rental Details Local End Time':'ended_at',
                        '03 - Rental Start Station Name':'start_station_name',
                        '03 - Rental Start Station ID':'start_station_id',
                        '02 - Rental End Station Name':'end_station_name',
                        '02 - Rental End Station ID':'end_station_id',
                        'User Type':'member_casual'}, inplace=True)

### Convert datatypes

In [10]:
#convert ride_id and rideable_type to character so they are uniform when stacking
q4_2019['ride_id'] = q4_2019['ride_id'].astype(str)
q4_2019['rideable_type'] = q4_2019['rideable_type'].astype(str)

In [11]:
q3_2019['ride_id'] = q3_2019['ride_id'].astype(str)
q3_2019['rideable_type'] = q3_2019['rideable_type'].astype(str)

In [12]:
q2_2019['ride_id'] = q2_2019['ride_id'].astype(str)
q2_2019['rideable_type'] = q2_2019['rideable_type'].astype(str)

### Combining 4 smaller datasets into one big data frame

In [13]:
alltrips = pd.concat([q2_2019,q3_2019,q4_2019,q1_2020],ignore_index=True)

### Removing unwanted columns (some data collecting was dropped in 2020)


In [14]:
alltrips = alltrips.drop(columns=['01 - Rental Details Duration In Seconds Uncapped','05 - Member Details Member Birthday Year','gender','birthyear','start_lat','start_lng','end_lat','end_lng','Member Gender','tripduration'])

## STEP 2: PROCESS 
### clean up and data manipulation

In [15]:
# in 'member_casual' column, Subscriber=Member and Customer=casual (so lets replacek them to keep only 2 instead of 4)

alltrips['member_casual'] = alltrips['member_casual'].replace('Subscriber','member')
alltrips['member_casual'] = alltrips['member_casual'].replace('Customer','casual')

In [16]:
#all three new columns: convert the 'started_at' column to date datatype 'Date' and extract --> day, month, day of week

alltrips['date'] = pd.to_datetime(alltrips['started_at'])

alltrips['day'] = alltrips['date'].dt.date
alltrips['month'] = alltrips['date'].dt.month
alltrips['year'] = alltrips['date'].dt.year
alltrips['day_of_week'] = alltrips['date'].dt.day_name()

In [17]:
#add a 'ride_length' column in alltrips that calculates ride length (in seconds) of each rides using 'started_at' and 'ended_at'
 
start_time = pd.to_datetime(alltrips['started_at'])
end_time = pd.to_datetime(alltrips['ended_at'])
alltrips['ride_length'] = (end_time - start_time).dt.total_seconds()

In [18]:
#Convert "ride_length" to numeric so we can run calculations on the data

alltrips['ride_length'] = alltrips['ride_length'].astype(int)

In [19]:
#some entries in 'ride_length' contains negative values, and some start station names contains "HQ QR" values which means these rows have to be removed as this case is when the bikes are taken out of docks and checked for quality

conditions = (alltrips['ride_length'] < 0) | (alltrips['start_station_name'] == "HQ_QR")
alltrips_v2 = alltrips.drop(alltrips[conditions].index)

In [20]:
len(alltrips.index)

3879822

In [21]:
len(alltrips_v2.index)

3879692

## STEP 3: ANALYZE 


In [32]:
#Grouping members into members/casual and finding out mean ride_length in both and assigning it to a new dataframe
resulta = (
        alltrips_v2
        .groupby(['member_casual','day_of_week'],observed=True)['ride_length']
        .mean()
        .reset_index()
)

In [37]:
resulta

,member_casual,day_of_week,ride_length
0,casual,Sunday,3581.405388
1,casual,Monday,3335.644620
2,casual,Tuesday,3569.798638
3,casual,Wednesday,3691.020290
4,casual,Thursday,3660.293285
5,casual,Friday,3758.220953
6,casual,Saturday,3331.913841
7,member,Sunday,919.974616
8,member,Monday,842.572569
9,member,Tuesday,826.142747


In [38]:
#Since the week names are not ordered from Sun-Sat, we used categorical function from pandas to order it (for analysis purposes)
custom_order = ['Sunday','Monday','Tuesday','Wednesday','Thursday','Friday','Saturday']
alltrips_v2['day_of_week'] = pd.Categorical(alltrips_v2['day_of_week'], categories=custom_order)

### Analyzing ridership data by type and week


In [39]:
#alltrips_v2['started_at'] = pd.to_datetime(alltrips_v2['started_at'])
#alltrips_v2['weekday'] = alltrips_v2['started_at'].dt.day_name()

result_1 = (
    alltrips_v2
    .groupby(['member_casual','day_of_week'],observed=True)
    .agg(number_of_rides=('ride_length','count'), average_duration=('ride_length','mean')).reset_index()
    .sort_values(by=['member_casual','day_of_week'])
)

In [42]:
result_1

,member_casual,day_of_week,number_of_rides,average_duration
0,casual,Sunday,181293,3581.405388
1,casual,Monday,104432,3335.644620
2,casual,Tuesday,91184,3569.798638
3,casual,Wednesday,93150,3691.020290
4,casual,Thursday,103316,3660.293285
5,casual,Friday,122913,3758.220953
6,casual,Saturday,209543,3331.913841
7,member,Sunday,267965,919.974616
8,member,Monday,472196,842.572569
9,member,Tuesday,508445,826.142747


### PLOTTING rider type (casual/member) v/s Number of rides (grouped into days of week)

In [29]:
fig = px.bar(
    result_1,
    x='day_of_week',
    y='number_of_rides',
    color='member_casual',
    barmode='group',  # 'group' for dodged bars
    title='Number of Rides by Weekday and Member Type',
    labels={'number_of_rides': 'Number of Rides', 'day_of_week': 'Weekday'},
)

# Show the plot
fig.show()

### PLOTTING rider type (casual/member) v/s Average ride length (grouped into days of week)

In [43]:
fig = px.bar(
    result_1,
    x='day_of_week',
    y='average_duration',
    color='member_casual',
    barmode='group',  # 'group' for dodged bars
    title='Number of Rides by Weekday and Member Type',
    labels={'number_of_rides': 'Number of Rides', 'average_duration': 'Avg duration of rides'},
)

# Show the plot
fig.show()